In [ ]:
import yaml
import sys
import torch

import pyrootutils
from pytorch_lightning.utilities.deepspeed import convert_zero_checkpoint_to_fp32_state_dict
# import mplfonts

root = pyrootutils.setup_root(__vsc_ipynb_file__, dotenv=True, pythonpath=True)
sys.path.append('/data/deeplearning-all-in-one/src')

In [ ]:
# mplfonts.use_font()

In [ ]:
labels = open('/data/clean_raw_text/district_label_map.txt', 'r').readlines()
labels = [x.strip('\n').split(' ')[0] for x in labels]

In [ ]:
model_path = '/data/deeplearning-all-in-one/logs/train/runs/2022-08-03/18-17-12/checkpoints/epoch_009.ckpt'
save_path = '/data/deeplearning-all-in-one/logs/train/runs/2022-08-03/18-17-12/checkpoints/model.ckpt'

In [ ]:
device = 'cuda:4' if torch.cuda.is_available() else 'cpu'

In [ ]:
convert_zero_checkpoint_to_fp32_state_dict(model_path, save_path)

In [ ]:
from src.models.dist_classify import DistClassify

In [ ]:
model = DistClassify.load_from_checkpoint(save_path, hparams=yaml.safe_load(open('/data/deeplearning-all-in-one/logs/train/runs/2022-08-03/18-17-12/.hydra/config.yaml', 'r')))

In [ ]:
model = model.to(device)
model.eval()

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext', cache_dir='/data/.cache')

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_json('/data/clean_raw_text/all_data_cleaned.json', lines=True)

In [ ]:
zwjl = data[data.src.eq('政务交流')]

In [ ]:
len(zwjl)

In [ ]:
model(tokenizer(zwjl.sample(32).text.values.tolist(), max_length=200, padding='max_length', truncation=True, return_tensors='pt').to(device)).softmax(dim=-1)

In [ ]:
from tqdm import tqdm

all_logits = []
with torch.no_grad():
    for i in tqdm(range(0, len(zwjl), 100), desc='generating pseudo-labels'):
        texts = zwjl.iloc[i:i+100].text.values.tolist()
        logits = model(tokenizer(texts, max_length=200, padding='max_length', truncation=True, return_tensors='pt').to(device)).softmax(dim=-1)
        all_logits.append(logits.cpu().numpy())

In [ ]:
import numpy as np

In [ ]:
logits = np.concatenate(all_logits, axis=0)

In [ ]:
zwjl['logits'] = logits.tolist()

In [ ]:
zwjl['pseudo_label'] = zwjl.logits.apply(lambda x: np.array(x).argmax())

In [ ]:
zwjl.groupby(by='pseudo_label').apply(len).plot.pie(labels=labels, figsize=(6,6))

In [ ]:
jx = data[data.src.eq('江西')]

In [ ]:
len(jx)

In [ ]:
all_logits = []
with torch.no_grad():
    for i in tqdm(range(0, len(jx), 100), desc='generating pseudo-labels'):
        texts = jx.iloc[i:i+100].text.values.tolist()
        logits = model(tokenizer(texts, max_length=200, padding='max_length', truncation=True, return_tensors='pt').to(device)).softmax(dim=-1)
        all_logits.append(logits.cpu().numpy())

In [ ]:
logits = np.concatenate(all_logits, axis=0)

In [ ]:
jx['logits'] = logits.tolist()

In [ ]:
jx['pseudo_label'] = jx.logits.apply(lambda x: np.array(x).argmax())

In [ ]:
jx.groupby(by='pseudo_label').apply(len).plot.pie(labels=labels, figsize=(6,6))

In [ ]:
zwjl['label'] = zwjl.pseudo_label.apply(lambda x: labels[x])

In [ ]:
jx['label'] = jx.pseudo_label.apply(lambda x: labels[x])

In [ ]:
jx[['text', 'label']].sample(10)

In [ ]:
street_data = pd.read_json('/data/clean_raw_text/dataset/street_test.json', lines=True)

In [ ]:
street_data.img.apply(lambda x: len(x) > 0).sum()

In [ ]:
all_agu_data = pd.concat([zwjl, jx], axis=0)

In [ ]:
all_agu_data[['text', 'label']].sample(10)

In [ ]:
all_agu_data['label'] = all_agu_data.logits

In [ ]:
train_data = pd.read_json('/data/clean_raw_text/dataset/district_train.json', lines=True)

In [ ]:
mix_data = pd.concat([train_data, all_agu_data], axis=0)

In [ ]:
mix_data

In [ ]:
mix_data[['text', 'label']].to_json('/data/clean_raw_text/dataset/district_train_with_pseudo_label.json', lines=True, orient='records', force_ascii=False)